In [1]:
import pandas as pd
import os
import glob


NOTEBOOK_DIR = os.path.dirname(os.path.abspath('__file__' if '__file__' in dir() else '.'))

if os.path.basename(NOTEBOOK_DIR) == 'notebooks':
    PROJECT_ROOT = os.path.dirname(NOTEBOOK_DIR)
else:
    PROJECT_ROOT = NOTEBOOK_DIR

BASE_PATH = os.path.join(PROJECT_ROOT, 'data', 'raw')
PROCESSED_PATH = os.path.join(PROJECT_ROOT, 'data', 'processed')

FILES = {
    'customers': 'olist_customers_dataset.csv',
    'items': 'olist_order_items_dataset.csv',
    'payments': 'olist_order_payments_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'geolocation': 'olist_geolocation_dataset.csv',
    'reviews': 'olist_order_reviews_dataset.csv',
    'sellers': 'olist_sellers_dataset.csv',
    'orders': 'olist_orders_dataset.csv',
    'translation': 'product_category_name_translation.csv'
}

def load_dataset(name):
    path = os.path.join(BASE_PATH, FILES[name])
    try:
        df = pd.read_csv(path)
        print(f"✅ [{name.upper()}] Cargado: {df.shape[0]} filas, {df.shape[1]} columnas.")
        return df
    except FileNotFoundError:
        print(f"❌ Error: No se encontró {path}")
        return None

def process_dates(df_orders):
    date_cols = [
        'order_purchase_timestamp', 
        'order_approved_at', 
        'order_delivered_carrier_date', 
        'order_delivered_customer_date', 
        'order_estimated_delivery_date'
    ]
    for col in date_cols:
        df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')
    print("🕒 Fechas convertidas a datetime en 'orders'.")
    return df_orders

def translate_products(df_products, df_trans):
    df_merged = df_products.merge(df_trans, on='product_category_name', how='left')
    
    df_merged['product_category_name'] = df_merged['product_category_name_english'].fillna(df_merged['product_category_name'])
    
    df_merged = df_merged.drop(columns=['product_category_name_english'])
    
    print("text-transform: capitalize; Traducción de productos completada.")
    return df_merged

def audit_data(df, name):
    nulls = df.isnull().sum().sum()
    dupes = df.duplicated().sum()
    print(f"🔍 Auditoría [{name}]: Nulos totales={nulls} | Duplicados={dupes}")

def create_master_dataframe(orders, items, customers):
    master = orders.merge(items, on='order_id', how='left')
    
    master = master.merge(customers, on='customer_id', how='inner')
    
    return master


if __name__ == "__main__":
    print("--- INICIANDO INGESTIÓN ---")
    datasets = {name: load_dataset(name) for name in FILES.keys()}
    
    if datasets['orders'] is not None and datasets['items'] is not None and datasets['customers'] is not None:
        
        datasets['orders'] = process_dates(datasets['orders'])
        
        if datasets['products'] is not None and datasets['translation'] is not None:
            datasets['products'] = translate_products(datasets['products'], datasets['translation'])
            
        audit_data(datasets['orders'], 'orders')
        audit_data(datasets['items'], 'items')
        audit_data(datasets['customers'], 'customers')

        print("\n--- CREANDO MASTER DATAFRAME ---")
        df_master = create_master_dataframe(datasets['orders'], datasets['items'], datasets['customers'])
        
        print(f"🚀 MASTER CREADO. Dimensiones: {df_master.shape}")
        print("Muestra de columnas:", df_master.columns.tolist())
        
        if not os.path.exists(PROCESSED_PATH):
            os.makedirs(PROCESSED_PATH)
            
        output_file = os.path.join(PROCESSED_PATH, 'olist_transactional_master.csv')
        df_master.to_csv(output_file, index=False)
        print(f"💾 Archivo guardado en: {output_file}")
        
    else:
        print("CRITICAL ERROR: Faltan archivos clave (Orders, Items o Customers).")

--- INICIANDO INGESTIÓN ---
✅ [CUSTOMERS] Cargado: 99441 filas, 5 columnas.
✅ [ITEMS] Cargado: 112650 filas, 7 columnas.
✅ [PAYMENTS] Cargado: 103886 filas, 5 columnas.
✅ [PRODUCTS] Cargado: 32951 filas, 9 columnas.
✅ [GEOLOCATION] Cargado: 1000163 filas, 5 columnas.
✅ [REVIEWS] Cargado: 99224 filas, 7 columnas.
✅ [SELLERS] Cargado: 3095 filas, 4 columnas.
✅ [ORDERS] Cargado: 99441 filas, 8 columnas.
✅ [TRANSLATION] Cargado: 71 filas, 2 columnas.
🕒 Fechas convertidas a datetime en 'orders'.
text-transform: capitalize; Traducción de productos completada.
🔍 Auditoría [orders]: Nulos totales=4908 | Duplicados=0
🔍 Auditoría [items]: Nulos totales=0 | Duplicados=0
🔍 Auditoría [customers]: Nulos totales=0 | Duplicados=0

--- CREANDO MASTER DATAFRAME ---
🚀 MASTER CREADO. Dimensiones: (113425, 18)
Muestra de columnas: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_del